In [120]:
from PyPDF2 import PdfFileWriter, PdfFileReader
import io
import subprocess
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase.pdfmetrics import registerFont
from reportlab.pdfbase.ttfonts import TTFont

In [121]:
from PyPDF2 import PdfFileWriter, PdfFileReader
import io
import subprocess
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase.pdfmetrics import registerFont
from reportlab.pdfbase.ttfonts import TTFont
registerFont(TTFont('Calibri', 'Calibri Regular.ttf')) # Just some font imports

In [122]:
def PDFcreator(name, surname, date_of_birth, gender, original="original.pdf", destination="destination.pdf"):
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=letter)
    can.setFont("Calibri", 11)
    can.drawString(158, 620, name+" " +surname)
    can.drawString(360, 620, gender)
    can.drawString(360, 606, date_of_birth)
    can.save()

    # move to the beginning of the StringIO buffer
    packet.seek(0)

    # create a new PDF with Reportlab
    new_pdf = PdfFileReader(packet)
    # read your existing PDF
    existing_pdf = PdfFileReader(open(original, "rb"))
    output = PdfFileWriter()
    # add the "watermark" (which is the new pdf) on the existing page
    page = existing_pdf.getPage(0)
    page.mergePage(new_pdf.getPage(0))
    output.addPage(page)
    # finally, write "output" to a real file
    outputStream = open(destination, "wb")
    output.write(outputStream)
    outputStream.close()

#     bashCommand = "open " + destination
#     process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
#     output, error = process.communicate()

In [123]:
PDFcreator("Harry", "Potter","31/07/1980","M","salary_slip_format_basic_edited.pdf","Auror_MoM.pdf")

In [124]:
import json
from pathlib import Path
from datetime import datetime

# date_obj = datetime.strptime(date_input, '%d-%b-%Y')
# print date_obj.strftime('%d-%m-%Y')

In [125]:
def fileiterator(filename):
    # Opening JSON file
    f = open(filename)
    # returns JSON object as a dictionary
    text_data = json.load(f)
    # Iterating through the json list
    data=text_data["_via_img_metadata"]
    for items in data:
        common_dict=dict()
        for attributes in data[items]["regions"]:
            common_dict[attributes["region_attributes"]["field_name"]]=attributes["region_attributes"]["value"]
        dob=(common_dict["birth_date"])

#         dob=dob.replace("Okt","Oct")#grc_passport
#         date_obj=datetime.strptime(dob, '%d %b %y')#grc_passport
#         dob=date_obj.strftime('%d/%m/%Y')#grc_passport

#         dob=dob.replace("Okt","Oct")
        date_obj=datetime.strptime(dob, '%d. %m. %Y')
        dob=date_obj.strftime('%d/%m/%Y')

        gender=common_dict["gender"]#.split("/")[1]
        
        name=common_dict["name"].title()
        
        surname=common_dict["surname"].title()#+" "+common_dict["surname_second"].title()
        
        print(name+" "+surname+" "+gender+" "+dob)

        foldername=filename.split(".")[0]
        
        Path(foldername).mkdir(parents=True, exist_ok=True)
        
        itemname=items.split(".")[0]
        
        print(itemname)
        PDFcreator(name, surname,dob,gender,"salary_slip_format_basic_edited.pdf",foldername+"/"+itemname+".pdf")
    f.close()

# fileiterator("templates/annotations/alb_id.json")
# fileiterator("templates/annotations/aze_passport.json") #gender contains prefix, date of birth has . instead of / ??
# fileiterator("templates/annotations/esp_id.json") # 2 surnames, date with spaces
# fileiterator("templates/annotations/est_id.json") # gender contains prefix
# fileiterator("templates/annotations/fin_id.json") # dob split by " " and .
# fileiterator("templates/annotations/grc_passport.json") # weird style for October and date nmonth year, and names as well!
# fileiterator("templates/annotations/lva_passport.json") # date of birth was strange - . after year!
# fileiterator("templates/annotations/rus_internalpassport.json")# weird names!
# fileiterator("templates/annotations/srb_passport.json") # gender split by spaces and slashes
# fileiterator("templates/annotations/svk_id.json")